In [1]:
import xarray as xr
from netCDF4 import Dataset, date2num
from datetime import datetime, date

In [2]:
with xr.open_mfdataset('cpc/*nc') as dset:
    print(dset)

<xarray.Dataset>
Dimensions:  (lat: 360, lon: 720, time: 3653)
Coordinates:
  * lat      (lat) float32 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75
  * lon      (lon) float32 0.25 0.75 1.25 1.75 2.25 ... 358.2 358.8 359.2 359.8
  * time     (time) datetime64[ns] 2011-01-01 2011-01-02 ... 2020-12-31
Data variables:
    precip   (time, lat, lon) float32 dask.array<chunksize=(365, 360, 720), meta=np.ndarray>
Attributes:
    Conventions:    CF-1.0
    version:        V1.0
    history:        created 9/2016 by CAS NOAA/ESRL PSD
    title:          CPC GLOBAL PRCP V1.0 RT
    dataset_title:  CPC GLOBAL PRCP V1.0
    Source:         ftp://ftp.cpc.ncep.noaa.gov/precip/CPC_UNI_PRCP/
    References:     https://www.psl.noaa.gov/data/gridded/data.cpc.globalprec...


### Correção das coordenadas ...

In [3]:
dset = dset.reindex(lat=dset['lat'][::-1])

/home/funceme/.local/lib/python3.7/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [4]:
dset.lat.values

array([-89.75, -89.25, -88.75, -88.25, -87.75, -87.25, -86.75, -86.25,
       -85.75, -85.25, -84.75, -84.25, -83.75, -83.25, -82.75, -82.25,
       -81.75, -81.25, -80.75, -80.25, -79.75, -79.25, -78.75, -78.25,
       -77.75, -77.25, -76.75, -76.25, -75.75, -75.25, -74.75, -74.25,
       -73.75, -73.25, -72.75, -72.25, -71.75, -71.25, -70.75, -70.25,
       -69.75, -69.25, -68.75, -68.25, -67.75, -67.25, -66.75, -66.25,
       -65.75, -65.25, -64.75, -64.25, -63.75, -63.25, -62.75, -62.25,
       -61.75, -61.25, -60.75, -60.25, -59.75, -59.25, -58.75, -58.25,
       -57.75, -57.25, -56.75, -56.25, -55.75, -55.25, -54.75, -54.25,
       -53.75, -53.25, -52.75, -52.25, -51.75, -51.25, -50.75, -50.25,
       -49.75, -49.25, -48.75, -48.25, -47.75, -47.25, -46.75, -46.25,
       -45.75, -45.25, -44.75, -44.25, -43.75, -43.25, -42.75, -42.25,
       -41.75, -41.25, -40.75, -40.25, -39.75, -39.25, -38.75, -38.25,
       -37.75, -37.25, -36.75, -36.25, -35.75, -35.25, -34.75, -34.25,
      

In [5]:
dset['lon_aux'] = xr.where(
    dset['lon'] > 180,
    dset['lon'] - 360,
    dset['lon']
)

dset = (
    dset
        .swap_dims({'lon': 'lon_aux'})
        .sel(**{'lon_aux': sorted(dset.lon_aux)})
        .drop('lon')
)

dset = dset.rename({'lon_aux': 'lon'})

/home/funceme/.local/lib/python3.7/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [6]:
dset.lon.values

array([-179.75, -179.25, -178.75, -178.25, -177.75, -177.25, -176.75,
       -176.25, -175.75, -175.25, -174.75, -174.25, -173.75, -173.25,
       -172.75, -172.25, -171.75, -171.25, -170.75, -170.25, -169.75,
       -169.25, -168.75, -168.25, -167.75, -167.25, -166.75, -166.25,
       -165.75, -165.25, -164.75, -164.25, -163.75, -163.25, -162.75,
       -162.25, -161.75, -161.25, -160.75, -160.25, -159.75, -159.25,
       -158.75, -158.25, -157.75, -157.25, -156.75, -156.25, -155.75,
       -155.25, -154.75, -154.25, -153.75, -153.25, -152.75, -152.25,
       -151.75, -151.25, -150.75, -150.25, -149.75, -149.25, -148.75,
       -148.25, -147.75, -147.25, -146.75, -146.25, -145.75, -145.25,
       -144.75, -144.25, -143.75, -143.25, -142.75, -142.25, -141.75,
       -141.25, -140.75, -140.25, -139.75, -139.25, -138.75, -138.25,
       -137.75, -137.25, -136.75, -136.25, -135.75, -135.25, -134.75,
       -134.25, -133.75, -133.25, -132.75, -132.25, -131.75, -131.25,
       -130.75, -130

In [7]:
# Calcula acumulado mensal
pr_mon = dset['precip'].resample(time='1MS').sum('time')
pr_mon

<xarray.DataArray 'precip' (time: 120, lat: 360, lon: 720)>
dask.array<stack, shape=(120, 360, 720), dtype=float32, chunksize=(1, 360, 720), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2011-01-01 2011-02-01 ... 2020-12-01
  * lat      (lat) float32 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon      (lon) float32 -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8

In [8]:
pr_mon.shape

(120, 360, 720)

In [9]:
# função retorna objeto datetime
def add_months(d, i):
    # d: objeto.date
    # i: incremento
    newmonth = (((d.month - 1) + i ) % 12 ) + 1
    newyear = int(d.year + ((( d.month - 1) + i ) / 12))
    return datetime(newyear, newmonth, d.day)

In [10]:
# https://unidata.github.io/netcdf4-python/

fname = '/home/funceme/my_notebooks/precip.cpc.mon.total.2011-2020.nc'             # Nome do arquivo NetCDF

ncfile = Dataset(fname, 'w', format='NETCDF4_CLASSIC')  # Abre o NetCDF para escrita

ncfile.createDimension('time', None)
ncfile.createDimension('lat', 360)        # num pontos latitude
ncfile.createDimension('lon', 720)        # num pontos longitude

# float = 'f4', double = 'f8'

# Eixo usados: time, lat, lon

lats = ncfile.createVariable('lat', 'f4', ('lat'))
lats.units = 'degrees_north'
lats.long_name = 'latitude'               # Nome descritivo do eixo lat 
lats.axis = 'Y'
lats[:] = pr_mon.lat.values               # Numpy Array

lons = ncfile.createVariable('lon', 'f4', ('lon'))
lons.units = 'degrees_east'
lons.long_name = 'longitude'              # Nome descritivo do eixo lon
lons.axis = 'X'
lons[:] = pr_mon.lon.values               # Numpy Array

list_dates = []
date_ini = date(2011, 1, 1)                      # Data Inicial
for i in range(120):                             # Num de passos de tempo
    list_dates.append(add_months(date_ini, i))   # Append lista de objetos datetime
    
times = ncfile.createVariable('time', 'f4', ('time'))
times.units = 'hours since 1800-01-01 00:00:00'
times.axis = 'T'
times.calendar = 'standard'
times.long_name = 'time'
# date2num recebe um objeto datetime ou lista de objetos datetime
times[:] = date2num(list_dates, times.units)  # https://unidata.github.io/netcdf4-python/#date2num

var = ncfile.createVariable(
    'precip',                             # Nome da variável
    'f4',
    ('time', 'lat', 'lon'),
    zlib=True                             # Compressão no NetCDF
)
var.units = 'mm'                          # Unidade da variável
var.long_name = 'Precipitation'           # Nome descritivo da variável
var[:] = pr_mon.values                    # Numpy Array

ncfile.source = 'CPC NOAA PSL'            # Fonte dos dados

ncfile.description = 'CPC Monthly Total'  # Descrição dos dados

ncfile.comment = 'Try to recreate CPC'    # Comentários

today = datetime.today()
ncfile.history = 'Created: ' + today.strftime("%b %d, %Y")

# ncfile.close()                          # Ver última célula! 

In [11]:
# Verificação ...
print(list_dates)

[datetime.datetime(2011, 1, 1, 0, 0), datetime.datetime(2011, 2, 1, 0, 0), datetime.datetime(2011, 3, 1, 0, 0), datetime.datetime(2011, 4, 1, 0, 0), datetime.datetime(2011, 5, 1, 0, 0), datetime.datetime(2011, 6, 1, 0, 0), datetime.datetime(2011, 7, 1, 0, 0), datetime.datetime(2011, 8, 1, 0, 0), datetime.datetime(2011, 9, 1, 0, 0), datetime.datetime(2011, 10, 1, 0, 0), datetime.datetime(2011, 11, 1, 0, 0), datetime.datetime(2011, 12, 1, 0, 0), datetime.datetime(2012, 1, 1, 0, 0), datetime.datetime(2012, 2, 1, 0, 0), datetime.datetime(2012, 3, 1, 0, 0), datetime.datetime(2012, 4, 1, 0, 0), datetime.datetime(2012, 5, 1, 0, 0), datetime.datetime(2012, 6, 1, 0, 0), datetime.datetime(2012, 7, 1, 0, 0), datetime.datetime(2012, 8, 1, 0, 0), datetime.datetime(2012, 9, 1, 0, 0), datetime.datetime(2012, 10, 1, 0, 0), datetime.datetime(2012, 11, 1, 0, 0), datetime.datetime(2012, 12, 1, 0, 0), datetime.datetime(2013, 1, 1, 0, 0), datetime.datetime(2013, 2, 1, 0, 0), datetime.datetime(2013, 3, 1, 0

In [12]:
# Verificação ...
# https://tinyurl.com/3jk9ba3j
print(times[:])

[1849584. 1850328. 1851000. 1851744. 1852464. 1853208. 1853928. 1854672.
 1855416. 1856136. 1856880. 1857600. 1858344. 1859088. 1859784. 1860528.
 1861248. 1861992. 1862712. 1863456. 1864200. 1864920. 1865664. 1866384.
 1867128. 1867872. 1868544. 1869288. 1870008. 1870752. 1871472. 1872216.
 1872960. 1873680. 1874424. 1875144. 1875888. 1876632. 1877304. 1878048.
 1878768. 1879512. 1880232. 1880976. 1881720. 1882440. 1883184. 1883904.
 1884648. 1885392. 1886064. 1886808. 1887528. 1888272. 1888992. 1889736.
 1890480. 1891200. 1891944. 1892664. 1893408. 1894152. 1894848. 1895592.
 1896312. 1897056. 1897776. 1898520. 1899264. 1899984. 1900728. 1901448.
 1902192. 1902936. 1903608. 1904352. 1905072. 1905816. 1906536. 1907280.
 1908024. 1908744. 1909488. 1910208. 1910952. 1911696. 1912368. 1913112.
 1913832. 1914576. 1915296. 1916040. 1916784. 1917504. 1918248. 1918968.
 1919712. 1920456. 1921128. 1921872. 1922592. 1923336. 1924056. 1924800.
 1925544. 1926264. 1927008. 1927728. 1928472. 19292

In [13]:
ncfile.close()  # Fecha o NetCDF

### Abrir no Ferret para verificar ...